# Forest Cover Type Prediction

https://www.kaggle.com/c/forest-cover-type-prediction

## Problem

Predict the forest cover type (the predominant kind of tree cover) from stricly cartographic varaibles (as opposed to remotely sensed data). 

## Libraries

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf

# set seed for reproducibility
tf.set_random_seed(6)

## Helper Functions


In [17]:
TEST_FILENAME = 'test.csv'
TRAIN_FILENAME = 'train.csv'
SAMPLE_SUBMISSION_FILENAME = 'sample_submission.csv'

def get_data(filename, force_download=False):
    '''Download ZIP file and unpack the CSV'''
    
    urlpath = 'https://www.kaggle.com/c/3936/download/'+ filename + '.zip'
    filepath = './data/' + filename
    
    # load data via url 
    if force_download:
        # login to Kaggle and retrieve the data
        login_info = {'Username': 'username', 'Password': 'password'}
        r = requests.post(requests.get(urlpath), data=login_info, prefetch=False)
        # write data to local file 512 KB at a time
        f = open(filepath, 'w')
        for chunk in r.iter_content(chunk_size=512*1024): 
            if chunk:
                f.write(chunk)
        f.close()
    
    # load data from directory (default)
    return pd.read_csv(filepath)

In [ ]:
def parse_data(header, data):
    return 

## Load Data

In [18]:
test = get_data(TEST_FILENAME)
train = get_data(TRAIN_FILENAME)
sample = get_data(SAMPLE_SUBMISSION_FILENAME)

/Users/tiffapedia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,1023,1036,1040,1042,1063,1085,1098,1099,1120,1128,1135,1151,1152,1165,1167,1181,1192,1204,1205,1206,1207,1215,1225,1227,1229,1231,1233) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/tiffapedia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,10

## EDA

In [19]:
# look at the first 5 observations
train.head(5)

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,NaN,323011,3854,481,1975,1,...,99.0,NaN,99,NaN,99,NaN,99,NaN,99,NaN
1,1,2,26,NaN,8.0,268131,2441,344,1981,1,...,NaN,NaN,1,NaN,2,NaN,2,NaN,2,NaN
2,2,1,16,NaN,7.0,167581,754,143,1995,1,...,1.0,NaN,2,NaN,2,NaN,2,NaN,2,NaN
3,3,4,44,5.0,NaN,445071,5705,604,1980,1,...,NaN,NaN,2,NaN,2,NaN,99,NaN,99,NaN
4,4,4,43,NaN,6.0,436161,5645,592,1958,1,...,NaN,NaN,1,NaN,1,NaN,1,NaN,1,NaN


In [20]:
# look at any random 5 observations
train.sample(5)

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
16993,16993,1,16,5.0,NaN,165061,909,178,1997,0,...,3.0,NaN,3,NaN,3,NaN,3,NaN,3,NaN
513,513,4,41,NaN,6.0,416191,4368,533,1976,1,...,2.0,NaN,2,NaN,2,NaN,2,NaN,2,NaN
8310,8310,3,33,NaN,7.0,337241,3669,457,1956,1,...,1.0,NaN,99,NaN,99,NaN,99,NaN,99,NaN
15503,15503,1,16,4.0,NaN,164011,905,177,1951,0,...,3.0,NaN,3,NaN,3,NaN,3,NaN,3,NaN
9830,9830,3,33,NaN,7.0,337121,3641,454,1958,1,...,2.0,NaN,3,NaN,3,NaN,3,NaN,3,NaN


In [21]:
# look at the last 5 observations
train.tail(5)

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
18250,18250,1,16,NaN,7.0,167221,880,173,1965,0,...,2.0,NaN,2,NaN,2,NaN,2,NaN,2,NaN
18251,18251,3,34,3.0,NaN,343041,4100,511,1992,0,...,99.0,NaN,99,NaN,99,NaN,99,NaN,99,NaN
18252,18252,3,34,NaN,6.0,346191,4287,530,1980,0,...,1.0,NaN,1,NaN,1,NaN,1,NaN,1,NaN
18253,18253,1,14,NaN,6.0,146041,361,71,1978,0,...,1.0,NaN,1,NaN,1,NaN,1,NaN,2,NaN
18254,18254,1,16,NaN,6.0,166161,800,153,1984,1,...,1.0,NaN,3,NaN,3,NaN,3,NaN,3,NaN


In [22]:
# describe the dataset
train.describe()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,LN1B,LN2_1,LN2_2,LN2_3,LN2_4,GN1,GN2,GN3,GN4,GN5
count,18255.000000,18255.000000,18255.000000,5653.000000,12602.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,...,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,14230.000000,18255.000000,18255.000000,18255.000000,18255.000000
mean,9127.000000,2.371460,28.558313,3.138864,6.793525,291360.681676,8030.510600,352.038346,1978.073185,0.537113,...,2.397316,2.145330,2.151575,2.830731,2.835881,5.590654,6.763079,7.421912,8.925883,8.817365
std,5269.908918,1.130523,9.822629,1.361434,0.769568,98126.483799,22061.608061,179.744071,14.740675,0.498634,...,1.231487,1.407641,1.407021,1.594319,1.592863,18.210791,20.537281,21.752528,24.523345,24.413247
min,0.000000,1.000000,11.000000,1.000000,6.000000,111011.000000,96.000000,24.000000,1917.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4563.500000,1.000000,21.000000,2.000000,6.000000,216071.000000,949.000000,178.000000,1969.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,9127.000000,2.000000,31.000000,3.000000,7.000000,313011.000000,2902.000000,354.000000,1981.000000,1.000000,...,2.000000,1.000000,1.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,13690.500000,3.000000,34.000000,4.000000,7.000000,348181.000000,4609.000000,523.000000,1990.000000,1.000000,...,4.000000,3.000000,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,18254.000000,4.000000,44.000000,5.000000,8.000000,448051.000000,99999.000000,633.000000,2001.000000,1.000000,...,4.000000,5.000000,5.000000,5.000000,5.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [23]:
# idenitfy missing values
train.isnull()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,False,False,False,False,True,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
1,False,False,False,True,False,False,False,False,False,False,...,True,True,False,True,False,True,False,True,False,True
2,False,False,False,True,False,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
3,False,False,False,False,True,False,False,False,False,False,...,True,True,False,True,False,True,False,True,False,True
4,False,False,False,True,False,False,False,False,False,False,...,True,True,False,True,False,True,False,True,False,True
5,False,False,False,True,False,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
6,False,False,False,True,False,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
7,False,False,False,False,True,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
8,False,False,False,False,True,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True
9,False,False,False,True,False,False,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True


In [24]:
len(train.isnull())

18255

## Model 